In [1]:
#Import Required Modules
#If a module is missing users can open a new notebook and paste the following to pip install
# !pip install wget

#Users will need to reprogram the following:
#Cell 2 username
#Cell 4 outDir

import json
import getpass
import requests
import wget
import sys
import os

In [3]:
#Form JSON Object to store credentials to generate API Key.

#Enter Your EarthExplorer (ERS) Credentials.
username='YOUR USERNAME'
password = getpass.getpass()

login = r'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/login'

creds={"username": username,
    "password": password,
    "authType": "",
    "catalogId": "EE"}

json_data = {'jsonRequest' : json.dumps(creds)}

r = requests.post(login, data = json_data)
token = r.json()['data']

if token == None:
    print ("API Key Failed to Generate")
  
else:
    print("Your API Key Generated Successfully")
    print(token)

#If Key fails to generate please see documentmentation to decode error https://earthexplorer.usgs.gov/inventory/documentation/errors

········
Your API Key Generated Successfully
39bfe7c30a874ca2a07eb7cad826a052


In [23]:
#Here we create structure to search inventory for Landsat 8 Collection 1 Level 1 Scenes acquired between 01/01/2013 and 12/31/2018
# for path 26 row 27 where maximum cloud cover is less than 10%.

#Define dataset see Dataset Search Endpoint on the API test page for additional datasets contained in inventory.
for dataset in ['Landsat_8_C1']:
    print("Searching Dataset {}" .format(dataset))
    
   #Here a temporal filter is created, users can also define a minimum bounding rectangle (MBR) filter to define search extent.
    search={"datasetName": dataset,
            "temporalFilter": {
                "dateField": "search_date",
                "startDate": "2014-01-01",
                "endDate": "2018-12-31",

            },
           
            #Here the additional criteria filter to search scenes in path 26 row 27 additional dataset fields related to specific
            #datasets can be found by querying the dataset name within the Dataset Fields Enpoint on the APU test page.
            "additionalCriteria": {
            "filterType": "and", 
            "childFilters": [
                {"filterType":"value","fieldId":"20514","operand":"like","value":"26"},
                {"filterType":"value","fieldId":"20516","operand":"like","value":"27"}
                ]
            },
            #The maximum number of results returned as an JSON object is 10 scenes. Users must specify the maximum number of
            #results to suit their needs.
            "maxResults": "10",
            
            #Here a maximum acceptable cloud cover of 10% is specified.
            "maxCloudCover": "10",
            
            "apiKey": (token)
            }
    
    searchUrl='https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/search'

    request = requests.get(searchUrl, params={'jsonRequest':json.dumps(search)})

    resultsList = request.json()['data']['results']
    
    #Create list to hold enity ids for scenes
    order = []
    prodID=[]
    for res in resultsList:
        order.append(res['entityId'])
        prodID.append(res['displayId'])

#Check to see if query resulted in scenes being returned.
if len(order) == 0:
    sys.exit("No Scenes Matching Criteria Were Found, Program will terminate")
else:
    print("Your Scene List Contains:")
    print(prodID)

Searching Dataset Landsat_8_C1
Your Scene List Contains:
['LC08_L1TP_026027_20140216_20170307_01_T1', 'LC08_L1TP_026027_20140304_20170307_01_T1', 'LC08_L1TP_026027_20140608_20170305_01_T1', 'LC08_L1TP_026027_20140726_20170304_01_T1', 'LC08_L1TP_026027_20141115_20170303_01_T1', 'LC08_L1TP_026027_20141201_20170302_01_T1', 'LC08_L1TP_026027_20150814_20170226_01_T1', 'LC08_L1TP_026027_20150915_20170225_01_T1', 'LC08_L1TP_026027_20161104_20170219_01_T1', 'LC08_L1TP_026027_20161222_20170218_01_T1']


In [24]:
#Create routine to generate download URLs for desired product for enity ids from search query. Please consult Get Bulk Download
#product endpoint for available products per dataset. Here Level-1 Landsat Geotiff will be downloaded. The product is downloaded
#using wget.

for enityID in [order]:
  orderParms ={'datasetName':dataset,
          'products':'FR_REFL',
          'entityIds': order,
          "apiKey": (token)
          
          }
  
  downloadURL="https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/download"
  download= requests.get(downloadURL, params={'jsonRequest':json.dumps(orderParms)})
    
urlList = []
download_list = download.json()['data']
for item in download_list:
    urlList.append(item['url'])

#Users will need to specify a download directory on their machine.
outDir="c:/users/nroberts/desktop/script"
for URL in urlList:
    wget.download(URL, out=outDir, bar=None)
print("The Following Scenes have downloaded:")
print(prodID)
print(len(order), "Total Files Downloaded to", os.path.abspath(outDir))
with open("c:/users/nroberts/desktop/script/output.txt", "w") as resultFile:
   for line in prodID:
    resultFile.write(line)
    resultFile.write("\n")
print("Download Complete")

The Following Scenes have downloaded:
['LC08_L1TP_026027_20140216_20170307_01_T1', 'LC08_L1TP_026027_20140304_20170307_01_T1', 'LC08_L1TP_026027_20140608_20170305_01_T1', 'LC08_L1TP_026027_20140726_20170304_01_T1', 'LC08_L1TP_026027_20141115_20170303_01_T1', 'LC08_L1TP_026027_20141201_20170302_01_T1', 'LC08_L1TP_026027_20150814_20170226_01_T1', 'LC08_L1TP_026027_20150915_20170225_01_T1', 'LC08_L1TP_026027_20161104_20170219_01_T1', 'LC08_L1TP_026027_20161222_20170218_01_T1']
10 Total Files Downloaded to c:\users\nroberts\desktop\script
Download Complete
